<a href="https://colab.research.google.com/github/VAIKUNDATH/assignment1/blob/main/case_study_4_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding='unicode_escape')

In [8]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [9]:
dt=df.drop(['emotion_in_tweet_is_directed_at'],axis=1)

In [10]:
dt.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


## **Preprocessing**

In [11]:
dt.columns=["Text","Label"]
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [12]:
dt['Label'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: Label, dtype: int64

In [13]:
dt['Label']=dt['Label'].map({'No emotion toward brand or product':'neutral','Positive emotion':'positive','Negative emotion':'negative',"I can't tell":'no idea'})

In [14]:
dt['Label']=dt['Label'].map({'neutral':2,'positive':3,'negative':1,'no idea':0})

In [15]:
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3
2,@swonderlin Can not wait for #iPad 2 also. The...,3
3,@sxsw I hope this year's festival isn't as cra...,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3


In [16]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9093 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.2+ KB


In [17]:
dt=dt.dropna()

In [18]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9092 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 213.1+ KB


In [19]:
import gensim
dt['text_clean_gensim']=dt['Text'].apply(lambda x: gensim.utils.simple_preprocess(x))
dt.head()

,Text,Label,text_clean_gensim
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1,"[wesley, have, iphone, after, hrs, tweeting, a..."
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3,"[jessedee, know, about, fludapp, awesome, ipad..."
2,@swonderlin Can not wait for #iPad 2 also. The...,3,"[swonderlin, can, not, wait, for, ipad, also, ..."
3,@sxsw I hope this year's festival isn't as cra...,1,"[sxsw, hope, this, year, festival, isn, as, cr..."
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3,"[sxtxstate, great, stuff, on, fri, sxsw, maris..."


# **Fitting RNN model**

In [21]:
from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(dt['text_clean_gensim']))
tokenized_texts=tokenizer.texts_to_sequences(dt['text_clean_gensim'])

In [22]:
from keras.utils import pad_sequences
x=pad_sequences(tokenized_texts,maxlen=100)

In [23]:
len(tokenizer.word_index)

9283

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,dt['Label'].values,test_size=0.2)

In [26]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,SimpleRNN,Dropout

In [27]:
model=Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))

In [28]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1188352   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                1390      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 204       
                                                                 
Total params: 1,190,496
Trainable params: 1,190,496
Non-

In [30]:
model.fit(x_train,y_train,epochs=15,validation_split=0.1)

Epoch 1/15
205/205 [==============================] - 11s 44ms/step - loss: 1.0493 - accuracy: 0.5343 - val_loss: 0.8721 - val_accuracy: 0.6264
Epoch 2/15
205/205 [==============================] - 9s 43ms/step - loss: 0.8924 - accuracy: 0.6024 - val_loss: 0.8286 - val_accuracy: 0.6291
Epoch 3/15
205/205 [==============================] - 8s 39ms/step - loss: 0.7813 - accuracy: 0.6675 - val_loss: 0.8537 - val_accuracy: 0.6058
Epoch 4/15
205/205 [==============================] - 8s 41ms/step - loss: 0.6568 - accuracy: 0.7424 - val_loss: 0.8662 - val_accuracy: 0.6195
Epoch 5/15
205/205 [==============================] - 9s 42ms/step - loss: 0.5647 - accuracy: 0.7898 - val_loss: 0.9316 - val_accuracy: 0.6044
Epoch 6/15
205/205 [==============================] - 9s 43ms/step - loss: 0.4874 - accuracy: 0.8264 - val_loss: 0.9955 - val_accuracy: 0.5962
Epoch 7/15
205/205 [==============================] - 9s 42ms/step - loss: 0.4432 - accuracy: 0.8432 - val_loss: 1.0023 - val_accuracy: 0.633

In [31]:
y_pred=model.predict(x_test)

57/57 [==============================] - 2s 16ms/step


The model obtained has an accuracy of 89%

# **Testing of model**

In [32]:
test1="its very good"
test1=tokenizer.texts_to_sequences([test1])
test2=pad_sequences(test1,maxlen=100)
out=model.predict(test2)
out

1/1 [==============================] - 0s 30ms/step


array([[0.00542196, 0.02583818, 0.13588768, 0.8328522 ]], dtype=float32)

The probability is more for the 4th value with analysis Positive.